 데이터에서 상위 33,000개의 샘플만 사용해주세요
 
 

## 1. 정제, 정규화, 전처리 (영어, 프랑스어)

### 1-1. 구두점(Punctuation)을 단어와 분리

- 분리 전 : he is a Good boy!

- 분리 후 : he is a Good boy !

### 2. 소문자로 변환

- 변환 전 : he is a Good boy !

- 변환 후 : he is a good boy !

### 3. 띄어쓰기 단위로 토큰를 수행하세요.

- 토큰화 전 : 'he is a good boy !'

- 토큰화 후 : ['he', 'is', 'a', 'good', 'boy', '!']


## 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어기

- 입력 시퀀스 : ['', 'courez', '!']

- 레이블 시퀀스 : ['courez', '!', ']

## 3. 케라스의 토크나이저로 텍스트를 숫자로 바꾸기

 - 아래 링크의 2. 케라스 텍스트 전처리 참고
https://wikidocs.net/31766

- 영어와 프랑스어에 대한 토크나이저를 각각 생성하고, tokenizer.texts_to_sequences()를 사용하여 모든 샘플에 대해서 정수 시퀀스로 변환

## 4. 임베딩 층(Embedding layer) 사용하기
 - 입력이 되는 각 단어를 임베딩 층을 사용해서 벡터화 하기. 아래 링크의 1. 케라스 임베딩층 참고

 - https://wikidocs.net/33793 
 
 - 코드 예시)

```
from tensorflow.keras.layers import Input, mbedding, Masking

# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(단어장의 크기, 임베딩 벡터의 차원)(encoder_inputs)
encoder_lstm = LSTM(hidden state의 크기, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
```

#### 주의할 점은 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만,

디코더의 훈련 과정과 테스트 과정(예측 과정)에서의 임베딩 층은 동일해야 합니다!

## 5. 모델 구현하기

## 6. 모델 평가하기

### 1. 정제, 정규화, 전처리 (영어, 프랑스어)
### 1-1. 구두점(Punctuation)을 단어와 분리
분리 전 : he is a Good boy!

분리 후 : he is a Good boy !

In [13]:
import pandas as pd
import re   
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
num_samples = 33000
lines = lines[['eng', 'fra']][:33000] # 3.3만개 샘플 사용

sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
22452,You can't just go.,\t Tu ne peux pas simplement t'en aller comme ...
19375,I like it so much.,\t Je l'aime tant ! \n
28007,You're not dressed.,\t Vous n'êtes pas habillées. \n
20218,It has cooled off.,\t Ça s'est rafraîchi. \n
23078,Check your pockets.,\t Vérifiez vos poches ! \n


#### 함수 구현
1. 소문자 바꾸기 
2. 특수문자 제거

In [14]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()      # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?'!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    #sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

In [15]:
lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))
lines.sample(5)

,eng,fra
29994,i need to get a job,\t J'ai besoin d'un emploi. \n
7212,you're sleepy,\t Tu es endormi. \n
6864,we must speak,\t Il nous faut nous parler. \n
24299,i have a pain here,\t J'ai mal ici. \n
3321,can we do it ?,\t Pouvons-nous le faire ? \n


#### 1-2. 소문자로 변환
변환 전 : he is a Good boy !

변환 후 : he is a good boy !

#### 1-3. 띄어쓰기 단위로 토큰를 수행하세요. 
토큰화 전 : 'he is a good boy !'

토큰화 후 : ['he', 'is', 'a', 'good', 'boy', '!']


### 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어기
입력 시퀀스 : ['', 'courez', '!']

레이블 시퀀스 : ['courez', '!', ']

In [16]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    
    with open(file_path, "r") as lines:
        for i, line in enumerate(lines):

            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line_input = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_input = [w for w in ("<sos> " + tar_line).split()]
            tar_line_target = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line_input)
            decoder_input.append(tar_line_input)
            decoder_target.append(tar_line_target)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [17]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()

In [19]:
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go'], ['hi'], ['hi'], ['run', '!'], ['run', '!']]
[['<sos>', 'va', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut'], ['<sos>', 'cours', '!'], ['<sos>', 'courez', '!']]
[['va', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '<eos>'], ['cours', '!', '<eos>'], ['courez', '!', '<eos>']]


In [20]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

In [21]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

### 3. 케라스의 토크나이저로 텍스트를 숫자로 바꾸기

 - 아래 링크의 2. 케라스 텍스트 전처리 참고
https://wikidocs.net/31766

- 영어와 프랑스어에 대한 토크나이저를 각각 생성하고, tokenizer.texts_to_sequences()를 사용하여 모든 샘플에 대해서 정수 시퀀스로 변환

In [22]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4761, 프랑스어 단어 집합의 크기 : 7880


In [23]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_fra.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_fra.index_word # 훈련 후 결과 비교할 때 사용

In [24]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[ 8400 27351 27624 ... 22809   650  8712]


In [25]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

### 4. 임베딩 층(Embedding layer) 사용하기
입력이 되는 각 단어를 임베딩 층을 사용해서 벡터화 하기. 아래 링크의 1. 케라스 임베딩층 참고

https://wikidocs.net/33793

코드 예시)

```
from tensorflow.keras.layers import Input, mbedding, Masking

# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(단어장의 크기, 임베딩 벡터의 차원)(encoder_inputs)
encoder_lstm = LSTM(hidden state의 크기, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
주의할 점은 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만,
디코더의 훈련 과정과 테스트 과정(예측 과정)에서의 임베딩 층은 동일해야 합니다!
```

### 5. 모델 구현하기

In [26]:
n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [27]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [28]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [29]:
latent_dim = 50

In [30]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [31]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [32]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [33]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     238050      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     394000      input_2[0][0]                    
_______________________________________________________________________________________

In [34]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 50)

Epoch 1/50
233/233 [==============================] - 9s 39ms/step - loss: 3.1789 - acc: 0.6414 - val_loss: 1.9853 - val_acc: 0.7027
Epoch 2/50
233/233 [==============================] - 8s 33ms/step - loss: 1.8532 - acc: 0.7145 - val_loss: 1.8144 - val_acc: 0.7207
Epoch 3/50
233/233 [==============================] - 8s 33ms/step - loss: 1.7186 - acc: 0.7282 - val_loss: 1.7014 - val_acc: 0.7368
Epoch 4/50
233/233 [==============================] - 8s 33ms/step - loss: 1.6019 - acc: 0.7474 - val_loss: 1.6064 - val_acc: 0.7474
Epoch 5/50
233/233 [==============================] - 8s 34ms/step - loss: 1.5165 - acc: 0.7572 - val_loss: 1.5553 - val_acc: 0.7560
Epoch 6/50
233/233 [==============================] - 8s 33ms/step - loss: 1.4477 - acc: 0.7687 - val_loss: 1.4781 - val_acc: 0.7680
Epoch 7/50
233/233 [==============================] - 8s 33ms/step - loss: 1.3915 - acc: 0.7769 - val_loss: 1.4289 - val_acc: 0.7748
Epoch 8/50
233/233 [==============================] - 8s 32ms/step - 

### 6. 모델 평가하기

In [35]:
encoder_model = Model(encoder_inputs, encoder_states)

In [36]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [37]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [38]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [39]:
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [40]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  you aren't invited 
번역문 : vous n' tes pas invit s 
예측문 :  vous n' tes pas invit e 


원문 :  he is tall and lean 
번역문 : il est grand et mince 
예측문 :  il est d sol e 


원문 :  you're so sweet 
번역문 : vous tes tellement gentilles ! 
예측문 :  vous tes si bonne chance 


원문 :  i hate losing 
번역문 : je d teste avoir le dessous 
예측문 :  je d teste le chien 


원문 :  we're having lunch 
번역문 : nous sommes en train de d jeuner 
예측문 :  nous pr mes pr par s r 




In [41]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_test[seq_index]))
  print("번역문 :",seq2tar(decoder_input_test[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  can i go now ? 
번역문 : puis je partir maintenant ? 
예측문 :  puis je aller maintenant ? 


원문 :  just do your job 
번역문 : faites simplement votre travail ! 
예측문 :  fais ton travail ne peuvent ! 


원문 :  i'm not pretty 
번역문 : je ne suis pas jolie 
예측문 :  je ne suis pas si idiot 


원문 :  did anyone care ? 
번역문 : quiconque s'en est il souci ? 
예측문 :  quiconque est il l soit trouv ? 


원문 :  you were perfect 
번역문 : vous f tes parfaite 
예측문 :  vous avez t parfait 


